In [1]:
import ecc_tools as tools
import numpy as np
import pandas as pd
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
#import emachine as EM
from direct_info import direct_info
from ecc_tools import distance_restr

import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

from scipy.sparse import csr_matrix
from joblib import Parallel, delayed
import timeit

import matplotlib.pyplot as plt
%matplotlib inline

import sys
import numpy as np
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
import expectation_reflection as ER
from direct_info import direct_info
from joblib import Parallel, delayed

# import pydca for plmDCA
from pydca.plmdca import plmdca
from pydca.meanfield_dca import meanfield_dca
from pydca.sequence_backmapper import sequence_backmapper
from pydca.msa_trimmer import msa_trimmer
from pydca.contact_visualizer import contact_visualizer
from pydca.dca_utilities import dca_utilities

tools.hide_toggle()

In [13]:
# -*- coding: utf-8 -*-
# This code was copied from ProDy documentation.
# Title: Evolution Analysis — ProDy
# URL: http://prody.csb.pitt.edu/tutorials/evol_tutorial/msaanalysis.html#get-msa-data

from prody import *
from pylab import *
Pfam_keys = searchPfam('1k2a').keys()

fetchPfamMSA('PF00074')

msa = parseMSA('PF00074_full.sth')


# Read in Protein structure
data_path = './'
pfam_id = 'PF00186'
msa_outfile = data_path + pfam_id + '.fa'
ref_outfile = 'ref_'+pfam_id+'.fa'
# create MSATrimmer instance 
trimmer = msa_trimmer.MSATrimmer(
    msa_outfile, biomolecule='protein', 
    refseq_file=ref_outfile,
)

trimmed_data = trimmer.get_msa_trimmed_by_refseq(remove_all_gaps=True)

#write trimmed msa to file in FASTA format
trimmed_data_outfile = 'MSA_'+pfam_id+'_Trimmed.fa'
with open(trimmed_data_outfile, 'w') as fh:
    for seqid, seq in trimmed_data:
        fh.write('>{}\n{}\n'.format(seqid, seq))

tools.hide_toggle()

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> 1k2a downloaded (1k2a.pdb.gz)
@> PDB download via FTP completed (1 downloaded, 0 failed).
@> UniProt ID code RNAS2_HUMAN for 1k2a chain A will be used.
@> Retrieving Pfam search results: https://pfam.xfam.org/protein/RNAS2_HUMAN?output=xml
@> Pfam search completed in 1.78s.
@> Query '1k2a' matched 1 Pfam families.
@> Pfam MSA for PF00074 is written as PF00074_full.sth.


ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ImportError: numpy.core.multiarray failed to import

In [ ]:
# Compute DCA scores using Pseudolikelihood maximization algorithm

plmdca_inst = plmdca.PlmDCA(
    trimmed_data_outfile,
    'protein',
    seqid = 0.8,
    lambda_h = 1.0,
    lambda_J = 20.0,
    num_threads = 10,
    max_iterations = 500,
)
#print("properties of plmDCA object: \n",dir(plmdca_inst),"\n\n")

sorted_DI = plmdca_inst.compute_sorted_DI()

N = plmdca_inst.sequences_len
B = plmdca_inst.num_sequences
print("sorted_DI (list) has shape: ",np.array(sorted_DI).shape,"\nRows: all combinations of sequence postitions ie sequence len choose 2 ")
print("num sequences = %d , sequences len = %d\n\n"%(B,N))

print("Print top 10 pairs")
for site_pair, score in sorted_DI[:10]:
    print(site_pair, score)


# compute DCA scores summarized by Frobenius norm and average product corrected
plmdca_FN_APC = plmdca_inst.compute_sorted_FN_APC()
tools.hide_toggle()

In [ ]:
# compute DCA scores summarized by Frobenius norm and average product corrected
plmdca_FN_APC = plmdca_inst.compute_sorted_FN_APC()

plmdca_visualizer = contact_visualizer.DCAVisualizer('protein', pdb[ipdb,6], pdb[ipdb,5],
    refseq_file = ref_outfile,
    sorted_dca_scores = plmdca_FN_APC,
    linear_dist = 4,
    contact_dist = 8.0,
)

contact_map_data = plmdca_visualizer.plot_contact_map()
tp_rate_data = plmdca_visualizer.plot_true_positive_rates()